In [1]:
from canvasapi import Canvas, assignment
import canvasapi as cnv
from tokens import canvas_token, notion_token
from notion_client import Client
from datetime import datetime,timedelta
import pytz

In [2]:
nclient = Client(auth=notion_token)
API_URL = "https://osu.instructure.com/"
API_KEY = canvas_token
canvas = Canvas(API_URL,API_KEY)
kiran = canvas.get_current_user()
courses = [x for x in kiran.get_courses()]
for x in courses:
     if x.enrollment_term_id != 163:
          courses.remove(x)
db_id = "57c6c1fdb5b54861af2fb7787344b4fc"
subjects = {
     'AU22 CSE 2221 - SW 1: Components (35445)':{'id': '8d669188-3c5e-4fea-84e4-2cb3fcf74ca8','name': 'Software Engineering 2221','color': 'yellow'},
     'AU22 ENGR 1181.01 - Fund Engr 1 (7063)':{'id': '9ff6a39b-c700-4ab7-9d66-29d963f01e8f','name': 'Intro to Engineering','color': 'green'},
     'AU22 ARTSSCI 1100.01H - ASC College Survey (15218)':{'id': 'e18cd657-84af-4152-a783-be92991ca93b','name': 'Art Sci Survey','color': 'default'},
     'AU22 PHYSICS 1250H - Mechanics, Spc Rel (20281)':{'id': '6dcb6cd3-3d2c-46d5-a070-f6f8bdb6969c','name': 'Physics 1250H','color': 'purple'},
     'AU22 MATH 2153 - Calculus 3 (18401)':{'id': '85591cad-2e1b-44f7-b06f-05f45a081c95','name': 'Calc 2153','color': 'red'},
     'AU22 ARTSSCI 1138.07 - First-Year Seminar (26389)':{'id': 'f4a7698c-1bd1-45b5-8474-db25bad5eee3','name': 'Stoicism','color': 'gray'}}

In [3]:
def get_schema(assignment:assignment,subject_dict:dict,course:str):
    backlog = {'id': '1', 'name': 'Backlog', 'color': 'red'}
    week_backlog = {'id': '2', 'name': 'Week Backlog', 'color': 'yellow'} 
    status = ""
    if assignment.due_at_date.date()-datetime.today().date()<=timedelta(days=7):
        status = week_backlog
    else:
        status = backlog
    if course in subject_dict.keys():
        subject = subject_dict[course]
    else:
        subject = {'name':course}
    schema = {
    "Status":{"select": status},
    'Name': {"title": [{"type":"text","text":{"content":assignment.name}}]},
    "Subject":{"select":subject},
    "Due Date":{"date":{"start":datetime.strptime(assignment.due_at,"%Y-%m-%dT%H:%M:%S%z").astimezone(pytz.timezone('US/Eastern')).strftime("%Y-%m-%dT%H:%M:%S%z")}},
    "Link":{"url":assignment.html_url},
    "Time to complete":{"select":{'id': 'b261419e-254f-44b9-8788-1532a54646fb','name': 'About an hour','color': 'red'}}
    }
    return schema

In [4]:
def add_assignment(assignment:assignment,subject_dict:dict,course:str):
    return nclient.pages.create(**{"parent":{"type":"database_id","database_id":db_id},"properties":get_schema(assignment,subjects,course)})


In [5]:
def update_course(course:cnv.course.Course,subject_dict:dict):
    course_assignments = [y for y in kiran.get_assignments(course)]
    for x in course_assignments:
        if course.name in subject_dict.keys():
            subject = subject_dict[course.name]['name']
        else:
            subject = course.name
        if len(nclient.databases.query(**{"database_id":db_id,"filter":{"and":[{"property":"Name","title":{"contains":x.name}},{"property":"Subject","select":{"equals":subject}}]}})['results'])<1 and x.get_submission(kiran).submitted_at==None and x.due_at!=None:
            print(x)
            print(add_assignment(assignment=x,subject_dict=subject_dict,course=course.name)['properties']['Name'])

In [6]:
def update_all(courses:list,subject_dict:dict):
    backlog = {'id': '1', 'name': 'Backlog', 'color': 'red'}
    week_backlog = {'id': '2', 'name': 'Week Backlog', 'color': 'yellow'} 
    backloglist = nclient.databases.query(**{"database_id":db_id,"filter":{"property":"Status","select":{"equals":"Backlog"}}})['results']
    for page in backloglist:
        date = datetime.fromisoformat(page['properties']['Due Date']['date']['start'])
        if date.date()-datetime.today().date()<timedelta(days=7):
            nclient.pages.update(page['id'],**{'properties':{'Status':{'select':week_backlog}}})
    for x in courses:
        update_course(x,subject_dict)

In [7]:
update_all(courses=courses,subject_dict=subjects)

In [17]:
for x in kiran.get_courses():
    print(x.__dict__)

{'_requester': <canvasapi.requester.Requester object at 0x7fdadeb28190>, 'id': 132981, 'name': 'A+ Research: Where Do You Start? Nov 22', 'account_id': 473, 'uuid': 'Lge2h5HgjEj7TUTbbuoPwA4Aa7dKs3CXhI1JTQQD', 'start_at': '2022-10-31T03:59:00Z', 'start_at_date': datetime.datetime(2022, 10, 31, 3, 59, tzinfo=<UTC>), 'grading_standard_id': None, 'is_public': False, 'created_at': '2022-10-25T16:23:30Z', 'created_at_date': datetime.datetime(2022, 10, 25, 16, 23, 30, tzinfo=<UTC>), 'course_code': 'A+ 22', 'default_view': 'modules', 'root_account_id': 1, 'enrollment_term_id': 6, 'license': 'private', 'grade_passback_setting': None, 'end_at': '2022-12-03T04:59:00Z', 'end_at_date': datetime.datetime(2022, 12, 3, 4, 59, tzinfo=<UTC>), 'public_syllabus': False, 'public_syllabus_to_auth': False, 'storage_quota_mb': 750, 'is_public_to_auth_users': False, 'homeroom_course': False, 'course_color': None, 'friendly_name': None, 'apply_assignment_group_weights': False, 'calendar': {'ics': 'https://osu.i

In [303]:
def add_links(course:cnv.course.Course):
    course_assignments = [y for y in kiran.get_assignments(course)]
    for x in course_assignments:
        if x.get_submission(kiran).submitted_at == None and x.due_at!=None:
            print(x)
            page = nclient.databases.query(**{"database_id":db_id,"filter":{"property":"Name","title":{"contains":x.name}}})['results'][0]
            nclient.pages.update(page['id'],**{'properties':{'Link':{'url':x.html_url}}})

In [57]:
def update_times(course:cnv.course.Course):
    course_assignments = [y for y in kiran.get_assignments(course)]
    for x in course_assignments:
        if x.get_submission(kiran).submitted_at == None and x.due_at!=None:
            print(x)
            page = nclient.databases.query(**{"database_id":db_id,"filter":{"and":[{"property":"Name","title":{"contains":x.name}},{"property":"Subject","select":{"equals":subjects[course.name]['name']}}]}})['results'][0]
            nclient.pages.update(page['id'],**{'properties':{'Due Date':{"date":{"start":datetime.strptime(x.due_at,"%Y-%m-%dT%H:%M:%S%z").astimezone(pytz.timezone('US/Eastern')).strftime("%Y-%m-%dT%H:%M:%S%z")}}}})

In [48]:
nclient.databases.query(**{"database_id":db_id,"filter":{"and":[{"property":"Name","title":{"contains":x.name}},{"property":"Subject","select":{"equals":"s"}}]}})

{'object': 'list',
 'results': [],
 'next_cursor': None,
 'has_more': False,
 'type': 'page',
 'page': {}}

In [13]:
nclient.databases.retrieve(db_id)['properties']['Subject']['select']

{'options': [{'id': '8227a232-c5fb-4e01-b345-6d1d9eff890f',
   'name': 'TeachAids',
   'color': 'blue'},
  {'id': '1496d3ae-b03f-4e96-8c87-e27dc19bb84e',
   'name': 'Coding Classes',
   'color': 'pink'},
  {'id': '77f20dbb-81be-44ca-b9a9-a322f7683743',
   'name': 'Stock Price Model',
   'color': 'gray'},
  {'id': '37f659f6-cdfc-42f7-bfdf-e87820a36810',
   'name': 'Internships',
   'color': 'orange'},
  {'id': 'b1051ad1-d157-4b95-9fd8-32a95f82de20',
   'name': 'Work',
   'color': 'brown'},
  {'id': '7f0a2282-7d65-4860-8fa4-c63c8b6f32c1',
   'name': 'Gym',
   'color': 'gray'},
  {'id': '72ec38b3-60f0-464f-a62d-5cb340f6faf1',
   'name': 'Personal',
   'color': 'purple'},
  {'id': 'cf43a828-4236-485c-b586-7fc9bc6cf40c',
   'name': 'Xavier',
   'color': 'red'},
  {'id': '8d669188-3c5e-4fea-84e4-2cb3fcf74ca8',
   'name': 'Software Engineering 2221',
   'color': 'yellow'},
  {'id': '9ff6a39b-c700-4ab7-9d66-29d963f01e8f',
   'name': 'Intro to Engineering',
   'color': 'green'},
  {'id': 'e18cd

In [17]:
nclient.databases.query(**{"database_id":db_id,"filter":{"and":[{"property":"Name","title":{"contains":"ES Week 12"}},{"property":"Subject","select":{"equals":"Physics 1250H"}}]}})

{'object': 'list',
 'results': [{'object': 'page',
   'id': '2c1b26d8-8f78-447c-ad93-0a43c4de9dda',
   'created_time': '2022-11-02T20:50:00.000Z',
   'last_edited_time': '2022-11-02T21:24:00.000Z',
   'created_by': {'object': 'user',
    'id': '6c910c0f-1f0f-432e-9e79-324309e6457f'},
   'last_edited_by': {'object': 'user',
    'id': 'bcd9f059-dbe8-44c2-8252-3c94a76e3e36'},
   'cover': None,
   'icon': None,
   'parent': {'type': 'database_id',
    'database_id': '57c6c1fd-b5b5-4861-af2f-b7787344b4fc'},
   'archived': False,
   'properties': {'Due Date': {'id': '%3CL%5B%60',
     'type': 'date',
     'date': {'start': '2022-11-06T23:59:00.000-05:00',
      'end': None,
      'time_zone': None}},
    'Scheduled Date': {'id': 'F%3Dd%3A', 'type': 'date', 'date': None},
    'Link': {'id': 'F%7CBr',
     'type': 'url',
     'url': 'https://osu.instructure.com/courses/126865/assignments/3109496'},
    'Project': {'id': 'N%7DUj',
     'type': 'relation',
     'relation': [],
     'has_more': F

In [26]:
s1 = {
    'Name': {"title": [{"type":"text","text":{"content":"Test"}}]},
    "Subject":{"select":{"name":"Test"}},
    "Time to complete":{"select":{'id': 'b261419e-254f-44b9-8788-1532a54646fb','name': 'About an hour','color': 'red'}}
    }

In [27]:
nclient.pages.create(**{"parent":{"type":"database_id","database_id":db_id},"properties":s1})

{'object': 'page',
 'id': '04df65f8-be0e-4802-8f0f-e179cc09e114',
 'created_time': '2022-11-03T16:49:00.000Z',
 'last_edited_time': '2022-11-03T16:49:00.000Z',
 'created_by': {'object': 'user',
  'id': '6c910c0f-1f0f-432e-9e79-324309e6457f'},
 'last_edited_by': {'object': 'user',
  'id': '6c910c0f-1f0f-432e-9e79-324309e6457f'},
 'cover': None,
 'icon': None,
 'parent': {'type': 'database_id',
  'database_id': '57c6c1fd-b5b5-4861-af2f-b7787344b4fc'},
 'archived': False,
 'properties': {'Due Date': {'id': '%3CL%5B%60', 'type': 'date', 'date': None},
  'Scheduled Date': {'id': 'F%3Dd%3A', 'type': 'date', 'date': None},
  'Link': {'id': 'F%7CBr', 'type': 'url', 'url': None},
  'Project': {'id': 'N%7DUj',
   'type': 'relation',
   'relation': [],
   'has_more': False},
  'Subject': {'id': '%60QUQ',
   'type': 'select',
   'select': {'id': '540739ff-3149-4952-91a8-48b8e2148bb7',
    'name': 'Test',
    'color': 'purple'}},
  'Time to complete': {'id': 'i%3D%7Bc',
   'type': 'select',
   'sel